In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## Keras Initialization amd Model Preparation

In [3]:
import numpy as np
np.random.seed(42)

import tensorflow as tf
tf.set_random_seed(42)
# import keras as K
from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Permute
from keras.layers import LSTM, Dropout, CuDNNGRU, Bidirectional, Embedding 
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop, Adam, Adagrad, Adadelta, SGD
from keras.layers import Input, Flatten, Dense, Dropout, Convolution1D, Conv1D, \
                         MaxPool1D, Lambda, GlobalMaxPooling1D, GlobalAveragePooling1D, \
                         BatchNormalization, Activation, AveragePooling1D, Concatenate, concatenate

from sklearn.model_selection import train_test_split, cross_val_score, \
                                    KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.utils import np_utils, to_categorical
from keras.preprocessing import sequence
import keras
import math        
        
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

Using TensorFlow backend.


In [4]:
with K.tf.device('/GPU:0'):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.9
    session = tf.Session(config=config)
    K.set_session(session)
#     set_gpu_option("0", 0.4)

In [8]:
import os
import sys

import random
random.seed(42)

#os.environ["CUDA_VISIBLE_DEVICES"]="0"
curruser = os.environ.get('USER')
sys.path.insert(0, './src/')

import re
import string
from string import digits
import pandas as pd
import numpy as np
import joblib
import csv

from LSTMPreProc import preprocIO
from Bio import SeqIO
import smart_open
# from bpe import Encoder
import sentencepiece as spm
# from tokenizers import CharBPETokenizer, ByteLevelBPETokenizer, SentencePieceBPETokenizer
from tqdm import tqdm_notebook, tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from pylab import rcParams
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 12, 5

In [5]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.corpora import Dictionary
from gensim import corpora
from gensim.models import TfidfModel, KeyedVectors
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.test.utils import common_texts

In [7]:
# dnaSegPath = ('./data/dnaseg/mart_export.txt')
# prep = preprocIO()
# seqArr = []
# for seq in tqdm_notebook(prep.streamFastaIO(dnaSegPath)):
#      seqArr+=[seq]

In [6]:
def plot_metrics(history):
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    plt.figure()
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    
    
def confusion_matrix_heatmap(y_true, y_pred):
    '''
    Построение Confusion matrix (матрицы ошибок)

    Parameters
    ----------
    y_test: pandas.Series, numpy.array
        Целевая для обучающего набора
    y_pred: pandas.Series, numpy.array
        Значения целевой переменной, предсказанные классификатором
    '''
    rcParams['figure.figsize'] = 6, 4
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Greens')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()    

## Load Pretrained BPE Tokenizer 

In [8]:
sp = spm.SentencePieceProcessor()
sp.Load("models/BPE/sentpbpe.model")

True

In [19]:
_str = seqArr[0]
sp.EncodeAsPieces(_str)

['▁',
 'ATGG',
 'AG',
 'ATCCC',
 'TG',
 'TGCC',
 'TG',
 'TGC',
 'AGCC',
 'G',
 'TC',
 'T',
 'TGGC',
 'TGC',
 'GCC',
 'GC',
 'GCC',
 'TC',
 'GGCC',
 'CC',
 'G',
 'TTG',
 'CCC',
 'GG',
 'AC',
 'TTTC',
 'GGC',
 'G',
 'CCC',
 'GG',
 'AC',
 'GCC',
 'TC',
 'TTTG',
 'ACC',
 'AGC',
 'GC',
 'TTC',
 'GGC',
 'G',
 'AGG',
 'GGC',
 'TGC',
 'TGG',
 'AGGCC',
 'G',
 'AGC',
 'TGGC',
 'TGC',
 'GC',
 'TC',
 'TGCC',
 'CC',
 'ACC',
 'ACGC',
 'TC',
 'GCC',
 'CCC',
 'TAC',
 'TACC',
 'TGC',
 'GC',
 'GC',
 'ACCC',
 'AGCG',
 'TGGC',
 'GC',
 'TGCCC',
 'G',
 'TCG',
 'CCC',
 'AGG',
 'TGCC',
 'G',
 'ACGG',
 'ACCCC',
 'GGCC',
 'AC',
 'TT',
 'TTC',
 'GG',
 'TGC',
 'TGC',
 'T',
 'AGAC',
 'G',
 'TG',
 'AAGC',
 'AC',
 'TTC',
 'TC',
 'GCC',
 'GG',
 'AGGAA',
 'AT',
 'TGC',
 'TGTC',
 'AAGG',
 'TGG',
 'TGGGC',
 'G',
 'AAC',
 'ACG',
 'TGG',
 'AGG',
 'TGC',
 'ACGC',
 'GC',
 'GCC',
 'ACG',
 'AGG',
 'AGCG',
 'CCC',
 'GG',
 'ATG',
 'AGC',
 'ACGG',
 'ATTC',
 'G',
 'TC',
 'GC',
 'GC',
 'GCG',
 'AG',
 'TTCC',
 'ACCG',
 'TC',
 'GC',

## Train Gensim FastText

In [32]:
# Iterator for FastText train
class GensimCorporaIter(object):
    def __init__(self, pathtofile, sp):
        self.pathtofile = pathtofile
        self.sp = sp
    def __iter__(self):
        path = datapath(self.pathtofile)  # absolute path to corpus
        with open(path, 'r', encoding='utf8', errors = 'ignore') as fin:
            for line in fin:
                line = line.rstrip('\n')
                yield list(self.sp.EncodeAsPieces(line))[1:]

In [35]:
dnaSegPathPost = os.path.join(os.getcwd(),'data/dnaseg/mart_export_proc.txt')
# GensimIter = GensimCorporaIter(dnaSegPathPost, sp)

In [36]:
model_gensim = FT_gensim(size=300, window=5, min_count=1, sg=1, negative=15, 
                         min_n=1, max_n=6, word_ngrams=1, workers=10)
model_gensim.build_vocab(sentences = GensimCorporaIter(dnaSegPathPost, sp))
model_gensim.train(sentences=GensimCorporaIter(dnaSegPathPost, sp), 
                   total_examples=model_gensim.corpus_count, epochs=20)

In [42]:
fw2vec = './models/fasttext/ft_300.vec'
model_gensim.wv.save_word2vec_format(fw2vec)
joblib.dump(model_gensim, './models/fasttext/ft_model.pkl', compress=('gzip',9))

['./models/fasttext/ft_model.pkl']

In [46]:
fw2vec = './models/fasttext/ft_300.vec'
w2vec = KeyedVectors.load_word2vec_format(fw2vec)

In [55]:
model_gensim.wv.similar_by_word('A')

[('ATG', 0.5819699764251709),
 ('AAAA', 0.5773802399635315),
 ('TGAC', 0.570457398891449),
 ('AACC', 0.5695433020591736),
 ('TGGCC', 0.5683084726333618),
 ('TGGGG', 0.5679406523704529),
 ('AAAC', 0.5669651031494141),
 ('AAGCC', 0.5583610534667969),
 ('ATGC', 0.5575288534164429),
 ('AAC', 0.5545167922973633)]

## Generate Embedding Matrix: `(Token's Id, Weighted Vector)`

In [5]:
import gzip
from collections import OrderedDict
import json

In [6]:
def weight_words2vec(pathtofile, pathtoexport, sp, fasttext_model):
    corpora = []
    result = []  
    print('Tokenize sequense corpora from file-->')
    path = datapath(pathtofile)  # absolute path to corpus
    with open(path, 'r', encoding='utf8', errors = 'ignore') as fin:
        for line in tqdm_notebook(fin):
            line = line.rstrip('\n')
            corpora.append(list(sp.EncodeAsPieces(line))[1:])
    
    maxlen = max(map(len, corpora))
    
    print("max tokens len: {}".format(maxlen))
    
    print("Build BOW and calculate TFIDF...")
    dct = Dictionary(corpora)
    bow = [dct.doc2bow(line) for line in corpora]
    modelTfIdf = TfidfModel(bow, smartirs='ntc')
    dic = [(k,v) for k,v in dct.items()]
    rev_dct = dict(map(reversed, dic))
    
    del dct, dic
    
    print("Calculate Weighted Vector for each token of interest-->")
    for i,line in tqdm_notebook(enumerate(corpora), total=len(corpora)):
        if len(line)!=0:
            embed = []
            weights = dict(modelTfIdf[bow[i]])
            embed = [fasttext_model[word]*weights[rev_dct[word]] for word in line]
#             mean_vec = np.array(embed).mean(axis=0)
            result.append(embed)
        else:
            pass
#             mean_vec = np.zeros((model_gensim.vector_size,))

#         if not np.mod(len(result), 10000) and len(result)!=0:
#             with gzip.open(pathtoexport, 'at') as fz:
#             with open(str(pathtoexport),'a', encoding='utf8') as fz: 
#                 writer = csv.writer(fz, delimiter=';', quotechar='"', lineterminator='\n')
#                 file_is_empty = os.stat(str(pathtoexport)).st_size == 0
#                 if file_is_empty:
#                     writer.writerow(['idx','vec'])
#                 for k,val in result.items():
#                     strarr = ['{0:.8f}'.format(ele) for ele in val]
#                     fz.write('{}: '.format(k)+' '.join(strarr)+'\n')
#                     writer.writerow([k,v])

#             result = OrderedDict()
    
    return maxlen, result

## Load Pretrained BPE Tokenizer 

In [9]:
sp = spm.SentencePieceProcessor()
sp.Load("models/BPE/sentpbpe.model")

True

## Calculate Weighted Embeddings ( This aproach is suitable only for the case of ad-hoc preparation of X matrix before feeding one on the input of Keras model)

In [7]:
dnaSegPathTrain = os.path.join(os.getcwd(),'data/dnaseg/csv/DnaSeg4TrainwLabelsSampled_wRandL.csv')
pathtoexport    = os.path.join(os.getcwd(),'models/embed/bpe_embeddings.json')

In [9]:
model_gensim = joblib.load('./models/fasttext/ft_model.pkl')

In [ ]:
maxlen, result = weight_words2vec(dnaSegPathTrain, pathtoexport, sp, model_gensim)

In [ ]:
word_seq_pad = sequence.pad_sequences(result, maxlen = maxlen, padding='post')

## Generate Embeding Matrix: Get FT Embeddings without TFIDF weighting

In [10]:
fw2vec = './models/fasttext/ft_300.vec'
w2vec = KeyedVectors.load_word2vec_format(fw2vec)

In [32]:
# Embed = w2vec.get_keras_embedding(train_embeddings=False)

Using TensorFlow backend.


In [11]:
embedding_matrix = np.zeros((sp.GetPieceSize()+1, w2vec.vector_size))
for i, vec in enumerate(w2vec.wv.vectors):
    embedding_matrix[sp.PieceToId(w2vec.index2word[i])] = vec

In [12]:
embedding_matrix.shape

(151, 300)

In [13]:
dnaSegPathTrain = os.path.join(os.getcwd(),'data/dnaseg/csv/DnaSeg4TrainwLabelsSampled_wRandL.csv')
data = pd.read_csv(dnaSegPathTrain, sep=';')

In [13]:
data.head()

,seq,label
0,ATATGTATTTTCTTTTTGTGGAGAGCATTTTTCCCTCGTGATTACA,0
1,TGGAGGGGAAAAGCCAGGAGACCTCCGAGCTTGCACATATTGTAGA,0
2,TTAACCTTGCAAGAACTCTTCAGGCACATATGGAAGATCTCG,0
3,TCTGAAGTAAATTATATCATTGAAAGACCAAGCTACCCTCTGAAGA...,0
4,GACTTGCATACCAACATAATCAGACCGTCTGCAGAAATTCTCCTAC...,0


### Prepare 1-st Input: Create one-hot encoded vectors using the `char_indices` map

In [14]:
SEQUENCE_LENGTH = data.seq.str.len().max()

sentences = data.seq.tolist()
next_chars = data.label.tolist()

chars = sorted(list(set(sentences[0])))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [15]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)),dtype=np.bool)
# y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences), total=len(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        #y[i, char_indices[next_chars[i]]] = 1

100%|██████████| 1865779/1865779 [01:10<00:00, 26525.99it/s]


In [16]:
y = to_categorical(next_chars, num_classes=4)

In [19]:
X.shape, y.shape

((2724312, 119, 4), (2724312, 4))

### Prepare 2-nd Input: Tensor with Tokens' Id to be passed on Embed Layer

In [17]:
def tokens_to_ids(encoder, x):
    x_toks = []
    for i in tqdm(range(len(x)), total=len(x)):
#         encoder.tokenize(x[i])
        x_toks.append(encoder.EncodeAsIds(x[i])[1:])
    
    max_len = max(map(len, x_toks))
    X_arr = np.zeros((len(x_toks), max_len), dtype='int32')
    for i, s in tqdm(enumerate(x_toks), total=len(x_toks)):
        X_arr[i, 0:len(s)] = s
    
    return X_arr

X_toks = tokens_to_ids(encoder=sp, x=data.seq.values)

100%|██████████| 1865779/1865779 [00:07<00:00, 243025.38it/s]


In [25]:
X_toks.shape

(1772490, 45)

In [ ]:
X_toks_train, X_toks_test, y_train, y_test = train_test_split(X_toks, y, random_state=42, shuffle=True, test_size=0.05)
X_train, X_test, _, _ = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.05)

In [ ]:
del X_toks, X

## Build and train model: Embed+BiLSTM+BatchNorm

In [28]:
def get_model(vocab_size, n_classes, seq_len_emb, seq_len_chars):
    
    inp = Input(shape =(seq_len_emb,), name='Input1')
    emb = Embedding(vocab_size, 300, name='BPE_emb')(inp)
    x = Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.1, 
                           return_sequences=False), name='BiLSTM_0')(emb)
    
#     inp_ = Reshape(target_shape=(seq_len_emb,1), name='Expand_Dims')(inp)
#     xlstm = Concatenate(name='Concat_LSTM')([x,inp_])

#     x = Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, 
#                            return_sequences=False), name='BiLSTM_1')(xlstm)    
    
    ###
    inp2 = Input(shape = (seq_len_chars, n_classes), name='Input2')
    y = Conv1D(kernel_size=5,
                filters=128,
                padding='valid',
                activation="relu",
                name='Conv1d_0')(inp2)
    y = Activation("relu")(y)
    y = MaxPool1D(pool_size=5, name='MaxPool1d-1')(y)
    y = BatchNormalization(name='BatchNorm-1')(y)
        
    y = Conv1D(kernel_size=4,
                filters=128,
                padding='valid',
                activation="relu",
                name='Conv1d_1')(inp2)
    y = Activation("tanh")(y)
    y = MaxPool1D(pool_size=4, name='MaxPool1d-1')(y)
    y = BatchNormalization(name='BatchNorm-1')(y)    
    
    y = Conv1D(kernel_size=4,
                filters=128,
                padding='valid',
                activation="tanh",
                name='Conv1d_2')(y)
    y = Activation("relu")(y)
    y = MaxPool1D(pool_size=4, name='MaxPool1d-2')(y)
    y = BatchNormalization(name='BatchNorm-2')(y)

    y = Dropout(0.3, name='Dropout_0.5')(y)
    y = Flatten(name='Flatten')(y)
    ###
    comb = Concatenate(name='Concat')([x,y])

    z = Dense(512, activation='relu', name='Dense_1024_0')(comb)
#     z = Dropout(0.5, name='Dropout_0.5_0')(z)
    z = Dense(512, activation='tanh', name='Dense_1024_1')(z) 
#     z = Dropout(0.5, name='Dropout_0.5_1')(z)
    z = Dense(n_classes, activation='softmax', name='Dense_NCLASS')(z)
    model = Model(inputs=[inp, inp2], outputs=z)
#     plot_model(model = model, to_file = './img/BiLSTM+emb-CNN.png', show_layer_names=False, show_shapes=True)
    model.summary()
    
    return model

## Compile Keras Model

In [29]:
VOCAB_SIZE = embedding_matrix.shape[0] #sp.GetPieceSize()+1 #embedding_matrix.shape[0]
MAXSEQLEN = X_toks_train.shape[1]
MAXCHARSLEN = X_train.shape[1]

model = get_model(vocab_size = VOCAB_SIZE, 
                  n_classes=len(chars), 
                  seq_len_emb=MAXSEQLEN, 
                  seq_len_chars=MAXCHARSLEN)

optimizer = SGD(lr=3e-3, momentum=0.1, decay=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input2 (InputLayer)             (None, 120, 4)       0                                            
__________________________________________________________________________________________________
Conv1d_1 (Conv1D)               (None, 117, 128)     2176        Input2[0][0]                     
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 117, 128)     0           Conv1d_1[0][0]                   
__________________________________________________________________________________________________
MaxPool1d-1 (MaxPooling1D)      (None, 29, 128)      0           activation_8[0][0]               
__________________________________________________________________________________________________
BatchNorm-

## If we wanna use Embedding Matrix with FT vectors then one should uncomment the following lines-->

In [30]:
embedding = model.get_layer('BPE_emb')
embedding.set_weights([embedding_matrix])
embedding.trainable = False

## Continue Training: Load Model from the last Checkpoint

In [22]:
path = './models/keras/bilstm_emb_fasttext.h5'
model = load_model(path)

In [24]:
model.weights

[<tf.Variable 'Conv1d_1_1/kernel:0' shape=(3, 4, 128) dtype=float32_ref>,
 <tf.Variable 'Conv1d_1_1/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm-1_1/gamma:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm-1_1/beta:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'Conv1d_2_1/kernel:0' shape=(3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'Conv1d_2_1/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm-2_1/gamma:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm-2_1/beta:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'BiLSTM_0_1/forward_lstm_9/kernel:0' shape=(300, 512) dtype=float32_ref>,
 <tf.Variable 'BiLSTM_0_1/forward_lstm_9/recurrent_kernel:0' shape=(128, 512) dtype=float32_ref>,
 <tf.Variable 'BiLSTM_0_1/forward_lstm_9/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'BiLSTM_0_1/backward_lstm_9/kernel:0' shape=(300, 512) dtype=float32_ref>,
 <tf.Variable 'BiLSTM_0_1/backward_lstm_9/recurrent_kernel:0' shape=(128, 512

In [31]:
path = './models/keras/bilstm_emb_fasttext.h5'

callbacks = [EarlyStopping(patience=12, monitor='val_loss'),
             ModelCheckpoint(path, monitor='val_loss', verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=1e-4, patience=3, verbose=1)]

BATCH_SIZE=256
n_epoch=160

history = model.fit([X_toks_train, X_train], y_train, batch_size=BATCH_SIZE, epochs=n_epoch, shuffle=True,
                     validation_split=0.15, verbose=1, callbacks=callbacks).history

Train on 1506616 samples, validate on 265874 samples
Epoch 1/160
1506616/1506616 [==============================] - 1049s 696us/step - loss: 1.3934 - acc: 0.2904 - val_loss: 1.3719 - val_acc: 0.3035

Epoch 00001: val_loss improved from inf to 1.37193, saving model to ./models/keras/bilstm_emb_fasttext.h5
Epoch 2/160
1506616/1506616 [==============================] - 1052s 698us/step - loss: 1.3739 - acc: 0.3009 - val_loss: 1.3690 - val_acc: 0.3059

Epoch 00002: val_loss improved from 1.37193 to 1.36899, saving model to ./models/keras/bilstm_emb_fasttext.h5
Epoch 3/160
1506616/1506616 [==============================] - 1052s 698us/step - loss: 1.3712 - acc: 0.3038 - val_loss: 1.3676 - val_acc: 0.3087

Epoch 00003: val_loss improved from 1.36899 to 1.36755, saving model to ./models/keras/bilstm_emb_fasttext.h5
Epoch 4/160
1506616/1506616 [==============================] - 1074s 713us/step - loss: 1.3696 - acc: 0.3058 - val_loss: 1.3668 - val_acc: 0.3097

Epoch 00004: val_loss improved fr

1506616/1506616 [==============================] - 1033s 686us/step - loss: 1.3654 - acc: 0.3115 - val_loss: 1.3650 - val_acc: 0.3122

Epoch 00035: val_loss did not improve

Epoch 00035: ReduceLROnPlateau reducing learning rate to 3.000000106112566e-11.
Epoch 36/160
1506616/1506616 [==============================] - 1035s 687us/step - loss: 1.3656 - acc: 0.3113 - val_loss: 1.3649 - val_acc: 0.3121

Epoch 00036: val_loss did not improve
Epoch 37/160
 944384/1506616 [=================>............] - ETA: 6:10 - loss: 1.3654 - acc: 0.3115

KeyboardInterrupt: 

In [ ]:
plot_metrics(history)

## OOB Validation

In [ ]:
model = load_model(path)

In [ ]:
y_pred = model.predict([X_toks_test, X_test])
y_pred = y_pred.argmax(axis=-1)

y_true = y_test.argmax(axis=-1)

In [ ]:
print(classification_report(y_true, y_pred))